<a href="https://colab.research.google.com/github/mgultekin/AiForMedicalDiagnosis/blob/main/U_NetOperations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install keras==2.3.1

In [3]:
pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 37kB/s 
     |████████████████████████████████| 512kB 37.7MB/s 
     |████████████████████████████████| 3.8MB 43.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=76a267197b90e67e6304cca421cc3913d10933544cf10cd246792582a038d76f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: 

In [4]:
#import libraries for building U-NET
import keras
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU
from keras.layers import Deconvolution3D
from keras.optimizers import Adam
from keras.layers.merge import concatenate
# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")

Using TensorFlow backend.


In [5]:
#defining an input layer tensor of shape
input_layer = Input(shape = (4, 160, 160, 16))


In [6]:
input_layer
# ? will be the batch size. The dims of the tensor are(batch_size,num_channels,height,width,length)

<tf.Tensor 'input_1:0' shape=(?, 4, 160, 160, 16) dtype=float32>

**Contracting(downward) path**

In [7]:
#lets define a conv3D tensor with 32 filters
down_depth_0_layer_0 = Conv3D(filters = 32,
                              kernel_size = (3,3,3),
                              padding = 'same',
                              strides = (1,1,1))(input_layer)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
down_depth_0_layer_0

<tf.Tensor 'conv3d_1/add:0' shape=(?, 32, 160, 160, 16) dtype=float32>

In [9]:
# now we are adding activation function
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0

<tf.Tensor 'activation_1/Relu:0' shape=(?, 32, 160, 160, 16) dtype=float32>

In [10]:
#depth 0, layer 1
down_depth_0_layer_1 = Conv3D(filters = 64,
                              kernel_size = (3,3,3),
                              padding = 'same',
                              strides = (1,1,1))(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1

<tf.Tensor 'activation_2/Relu:0' shape=(?, 64, 160, 160, 16) dtype=float32>

**Max pooling**

In [12]:
#next thing is defining max pooling after down conv
down_depth_0_layer_pool = MaxPooling3D(pool_size= (2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool

<tf.Tensor 'max_pooling3d_1/transpose_1:0' shape=(?, 64, 80, 80, 8) dtype=float32>

In [13]:
#depth 1, layer 0
down_depth_1_layer_0 = Conv3D(filters = 64,
                              kernel_size =(3,3,3),
                              padding = 'same',
                              strides = (1,1,1))(down_depth_0_layer_pool)
        
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<tf.Tensor 'activation_3/Relu:0' shape=(?, 64, 80, 80, 8) dtype=float32>

In [14]:
#depth 1 layer 1
down_depth_1_layer_1 = Conv3D(filters = 128,
                              kernel_size = (3,3,3),
                              padding = 'same',
                              strides = (1,1,1))(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<tf.Tensor 'activation_4/Relu:0' shape=(?, 128, 80, 80, 8) dtype=float32>

In [15]:
#we assume that depth of our model is 2, thats why there will be no more max pooling operation

**Expanding Path(upward)**

In [17]:
#add an upsamlpling operation to the network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0

<tf.Tensor 'up_sampling3d_1/concat_2:0' shape=(?, 128, 160, 160, 16) dtype=float32>

In [18]:
#now we're going to concatenate upsampled depth 0 with downsampled depth 0, both of them have same height,width,length
print(up_depth_0_layer_0)
print(down_depth_0_layer_1)

Tensor("up_sampling3d_1/concat_2:0", shape=(?, 128, 160, 160, 16), dtype=float32)
Tensor("activation_2/Relu:0", shape=(?, 64, 160, 160, 16), dtype=float32)


In [19]:
#lets add a concatenation along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,down_depth_0_layer_1],axis=1)
up_depth_1_concat

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 192, 160, 160, 16) dtype=float32>

In [21]:
up_depth_1_layer_1 = Conv3D(filters = 64,
                            kernel_size = (3,3,3),
                            padding = 'same',
                            strides = (1,1,1))(up_depth_1_concat)
up_depth_1_layer_1

<tf.Tensor 'conv3d_5/add:0' shape=(?, 64, 160, 160, 16) dtype=float32>

In [22]:
up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1

<tf.Tensor 'activation_5/Relu:0' shape=(?, 64, 160, 160, 16) dtype=float32>

In [23]:
up_depth_1_layer_2 = Conv3D(filters = 64,
                            kernel_size = (3,3,3),
                            padding = 'same',
                            strides = (1,1,1),
                            )(up_depth_1_layer_1)

up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)
up_depth_1_layer_2

<tf.Tensor 'activation_6/Relu:0' shape=(?, 64, 160, 160, 16) dtype=float32>

**Final Convolution**

In [24]:
final_conv = Conv3D(filters=3, #3categories
                    kernel_size = (1,1,1),
                    padding = 'valid',
                    strides = (1,1,1),
                    )(up_depth_1_layer_2)
final_conv

<tf.Tensor 'conv3d_7/add:0' shape=(?, 3, 160, 160, 16) dtype=float32>

In [25]:
final_activation = Activation('sigmoid')(final_conv)
final_activation

<tf.Tensor 'activation_7/Sigmoid:0' shape=(?, 3, 160, 160, 16) dtype=float32>

**Create and compile the model**

In [26]:
model = Model(inputs = input_layer, outputs = final_activation)
model.compile(optimizer = Adam(lr = 0.00001),
              loss = 'categorical_crossentropy',
              metrics = ['categorical_accuracy'])

In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 160, 160, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 160, 160, 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 160, 160, 55360       activation_1[0][0]               
____________________________________________________________________________________________